# LSST Image Service Tutorial

This notebook provides a simple tutorial where you will inspect the observations table,
and work with fits images of observations you select.

## 1. Create TAP client

We will use the TAP service to access the ivoa.ObsCore table, which contains observation
data.

In [ ]:
import lsst.afw.display
import lsst.afw.image
import lsst.rsp
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

lsst.afw.display.setDefaultBackend('matplotlib')
service = lsst.rsp.get_tap_service()

In [ ]:
# This cell is a hack until we get the following PR in:
# https://github.com/lsst-sqre/lsst-rsp/pull/11
service._session.add_security_method_for_url("https://data-int.lsst.cloud/api/datalink", "lsst-token")
service._session.add_security_method_for_url("https://data-int.lsst.cloud/api/cutout", "lsst-token")

## 2. Query the ObsCore table for an observation

Execute a SQL statement against the ObsCore table, where you can limit the observations by time,
type, and any other SQL features.

We will display in a table the rows returned, but for the rest of this notebook, we will only
work with the first row.

In [ ]:
results = service.search("SELECT * FROM ivoa.ObsCore WHERE dataproduct_subtype='lsst.calexp' LIMIT 1")
r = results[0]
results.to_table().show_in_notebook()

## 3. Access the Datalink links service

The Datalink links service, which is referred to in the access_url column in the table above,
provides all sorts of interesting information about a particular observation.

In [ ]:
print(f"Datalink link service url: {r.getdataurl()}")
dr = DatalinkResults.from_result_url(r.getdataurl(), session=service._session)
dr.to_table().show_in_notebook()

## 4. Use SODA to create a circle cutout

Since the full observation is very large, you can request a cutout of different shapes around the object
you are looking at.

The first line sets up a request to the SODA service.
The second line sets up the circle shape, providing RA, DEC, and a radius.
The last lines read the file from the network and write it to a local fits file.

In [ ]:
sq = SodaQuery.from_resource(dr, dr.get_adhocservice_by_id("cutout-sync"), session=service._session)
sq.circle = (r["s_ra"], r["s_dec"], 0.09)

with open('circle-cutout.fits', 'bw') as f:
    f.write(sq.execute_stream().read())

## 5. Display the cutout

In [ ]:
i = lsst.afw.image.ImageF('circle-cutout.fits')     #read FITS file into afw image object
afw_display = lsst.afw.display.Display()      #get an alias to the lsst.afw.display.Display() method
afw_display.scale('asinh', 'zscale')    #set the image stretch algorithm and range
afw_display.mtv(i)                     #load the image into the display